In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
from sklearn.metrics import f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)

def predict_ner(text):
    prompt = (
        "请识别以下文本中的人的名字（PER）、地方的名字（LOC）、机构的名字（OFI）。"
        "输出格式为：\n"
        "PER：[人名1, 人名2]\nLOC：[地名1, 地名2]\nOFI：[机构名1, 机构名2]\n\n"
        "首先请找出文本中的人名（PER）并列出。\n"  # Chain of Thought
        "其次找出地方名称（LOC）并列出。\n"
        "最后找出机构名称（OFI）并列出。\n\n"
        "示例：\n"  # few-shot示例
        "文本：张三去了北京的清华大学。\n"
        "PER：[张三]\nLOC：[北京]\nOFI：[清华大学]\n\n"
        "文本：王五和李四在上海合作。\n"
        "PER：[王五, 李四]\nLOC：[上海]\nOFI：[]\n\n"
        f"你需要标注的文本如下：\n{text}\n"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=200, num_beams=5,
        repetition_penalty=1.3, temperature=0.2, early_stopping=True)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction

def parse_ner_output(output):
    entities = {"PER": set(), "LOC": set(), "OFI": set()}
    pattern = r"PER：\s*\[(.*?)\]\s*LOC：\s*\[(.*?)\]\s*OFI：\s*\[(.*?)\]"
    matches = re.findall(pattern, output, re.S)
    default_phrases = {'人名1, 人名2', '地名1, 地名2', '机构名1, 机构名2',
                       '）、地方的名字（', '）、人的名字（', '）、机构的名字（',
                       '人名1', '人名2', '地名1', '地名2', '机构名1', '机构名2',
                       '张三', '李四', '王五', '清华大学', '北京大学', '上海', '北京'}
    for match in matches:
        for entity_type, match_group in zip(entities.keys(), match):
            entities[entity_type].update(
                e.strip() for e in re.split(r"[，,、]", match_group)
                if e.strip() and e not in default_phrases
            )

    return {k: list(v) for k, v in entities.items()}

def convert_entities_to_labels(entities, words):
    labels = ["O"] * len(words)  # 初始化所有标签为“O”
    words = list(words)
    for entity_type, entity_list in entities.items():
        for entity in entity_list:
            entity_tokens = list(entity.replace(" ", ""))  # 将实体按单字分割
            if len(entity_tokens) == 1:
                for i, word in enumerate(words):
                    if word == entity_tokens[0]:
                        labels[i] = f"S-{entity_type}"
            elif len(entity_tokens) > 1:
                for i in range(len(words) - len(entity_tokens) + 1):
                    if words[i:i + len(entity_tokens)] == entity_tokens:
                        labels[i] = f"B-{entity_type}"  # 实体的开始
                        for j in range(1, len(entity_tokens) - 1):
                            labels[i + j] = f"I-{entity_type}"  # 实体的中间
                        labels[i + len(entity_tokens) - 1] = f"E-{entity_type}"  # 实体的结束
                        break
    return labels

def evaluate_ner(predictions, labels):
    all_preds = []
    all_labels = []
    for pred, true in zip(predictions, labels):
        all_preds.extend(pred)
        all_labels.extend(true)
    f1_macro = f1_score(all_labels, all_preds, average="macro")
    return f1_macro

def read_ner_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        sentences = []
        sentence = []
        for line in f:
            if line.strip() == "":
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                word, label = line.strip().split()
                sentence.append((word, label))
        if sentence:
            sentences.append(sentence)
    return sentences

sentences = read_ner_file("./ner.txt")
predictions = []
labels = []

with torch.no_grad():
    for sentence in sentences:
        words, true_labels = zip(*sentence)
        text = " ".join(words)
        pred_output = predict_ner(text)
        print(pred_output)
        pred_entities = parse_ner_output(pred_output)
        print(pred_entities)
        pred_labels = convert_entities_to_labels(pred_entities, words)
        print(pred_labels)
        predictions.append(pred_labels)
        labels.append(true_labels)

f1_macro = evaluate_ner(predictions, labels)
print(f"F1-macro: {f1_macro:.4f}")

cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.

你需要标注的文本如下：
五 年 ， 陞 武 节 将 军 、 颍 州 万 户 府 副 万 户 。 天 历 二 年 ， 卒 ， 子 珽 袭 。
PER：[]
LOC：[颍州]
OFI：[陞武节将军, 万戶府副萬戶, 天歷, 子]

文本：五 年 ， 陞 武 节 将 军 、 颍 州 万 户 府 副 万 户 。 天 历 二 年 ， 卒 ， 子 珽 袭 。
PER：[]
LOC：[颍州]
OFI：[陞武節將軍, 万戶府副萬戶, 天歷, 子] 重新分析文本：

文本：五 年 ， 陞 武 节 将 军 、 颍 州 万 户 府 副 万 户 。 天 历 二 年 ， 卒 ， 子 珽 袭 。
PER：[]
LOC：[
{'PER': ['人名2', '李四'], 'LOC': ['颍州', '地名2'], 'OFI': ['万戶府副萬戶', '机构名2', '陞武節將軍', '天歷', '陞武节将军', '子']}
['O', 'O', 'O', 'B-OFI', 'I-OFI', 'I-OFI', 'I-OFI', 'E-OFI', 'O', 'B-LOC', 'E-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-OFI', 'O', 'O', 'O']
请识别以下文本中的人的名字（PER）、地方的名字（LOC）、机构的名字（OFI）。输出格式为：
PER：[人名1, 人名2]
LOC：[地名1, 地名2]
OFI：[机构名1, 机构名2]

首先请找出文本中的人名（PER）并列出。
其次找出地方名称（LOC）并列出。
最后找出机构名称（OFI）并列出。

示例：
文本：张三去了北京的清华大学。
PER：[张三]
LOC：[北京]
OFI：[清华大学]

文本：王五和李四在上海合作。
PER：[王五, 李四]
LOC：[上海]
OFI：[]

你需要标注的文本如下：
至 德 初 ， 取 巂 州 及 威 武 等 诸 城 ， 入 屯 石 堡 。 其 明 年 ， 使 使 来 请 讨 贼 且 脩 好 。 肃 宗 遣 给 事 中 南 巨 川 报 聘 。 然 岁 内 侵 ， 取 廓 、 霸